In [4]:
!pip install scikit-learn

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 31.4 MB/s eta 0:00:001m30.4 MB/s eta 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)



In [2]:
from tqdm import tqdm
import re
import pickle

In [3]:
def get_accuracy(outputs, labels):
    matches = sum(p == y for p, y in zip(outputs, labels))
    total = len(labels)
    return matches / total if total > 0 else 0.0

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/Colab Notebooks/Intern_summer2025/code

/content/drive/MyDrive/Colab Notebooks/Intern_summer2025/code


# Data

In [4]:
# dataset = pd.read_csv('../../data/dataset.csv')
# dataset = pd.read_csv('../../data/prompts-instruct-model.csv')
dataset = pd.read_csv('../../data/dataset2_sin_os_status.csv')

In [5]:
dataset.shape

(713310, 5)

In [6]:
print(dataset[:1].prompt.values[0])

Instruction: You are a helpful medical assistant. 
Based on the following patient data, answer the multiple-choice question by selecting the correct letter ([A], [B], [C], or [D]).
Context: This is a cancer patient with the following characteristics:
 * Age of the patient: 68.0
 * HER2 status: No
 * MSI Type: Indeterminate
 * Overal Survival Status: 0:LIVING
 * Cancer stage: Stage 1-3
 * Smoking history: Former/Current Smoker
 * Fraction Genome altered: 0.3146
 * History of PDL-1: No
 * Number of tumor diagnoses (ICD-O codes): 2
 * Mutation Count: 1.0
 * Gender: Female
 * Mutations history: PIK3C2G, FLT4, PIK3R3, BLM, MAP3K1, CCNE1, ATR, BRIP1, TSHR, SDHA, RET, PTPRD, FBXW7, NSD1, PDGFRA, AXIN2, TP53, SMARCA4, PTPRT, HGF, DOT1L, KDM5A, PTPRS, CREBBP, KMT2C, ATRX
 * Treatment history: CISPLATIN, ETOPOSIDE, CARBOPLATIN, INVESTIGATIONAL, INVESTIGATIONAL, NIVOLUMAB
Question: "Where did the cancer spread to, affecting the patient's health and treatment options?"
Options (choose ONLY one opt

There are 23777 patients in the dataset. We created 5 versions of a prompt per variable(7) for each patient. So 23777 x 5 x7 = 832195. That's 35 prompts per patient id

In [7]:
dataset.isnull().sum()

PATIENT_ID    0
variable      0
prompt        0
answer        0
replicate     0
dtype: int64

# Data Split by unique patients 80/10/10

no patient appears in more than one of the training, validation, or test sets


In [8]:
dataset.head()

,PATIENT_ID,variable,prompt,answer,replicate
0,P-0000012,METASTATIC_SITES,Instruction: You are a helpful medical assista...,[C],0
1,P-0000012,OS_MONTHS,Instruction: You are a helpful medical assista...,[C],0
2,P-0000012,CANCER_TYPE,Instruction: You are a helpful medical assista...,[B],0
3,P-0000012,TMB_nonsynonymous,Instruction: You are a helpful medical assista...,[D],0
4,P-0000012,Tumor_Purity,Instruction: You are a helpful medical assista...,[A],0


In [9]:


#Get unique patient IDs
unique_patients = dataset['PATIENT_ID'].unique()

np.random.shuffle(unique_patients)

#Split into train (80%), temp (20%)
train_patients, temp_patients = train_test_split(
    unique_patients,
    test_size=0.2,
    random_state=42
)

#Split temp into validation (10%) and test (10%)
val_patients, test_patients = train_test_split(
    temp_patients,
    test_size=0.5,
    random_state=42
)

#Create splits based on patient IDs
train_df = dataset[dataset['PATIENT_ID'].isin(train_patients)].copy()
val_df   = dataset[dataset['PATIENT_ID'].isin(val_patients)].copy()
test_df  = dataset[dataset['PATIENT_ID'].isin(test_patients)].copy()


In [10]:
train_df.shape, val_df.shape, test_df.shape

((570630, 5), (71340, 5), (71340, 5))

In [11]:
X_train = train_df['prompt']
y_train = train_df['answer']
X_val = val_df['prompt']
y_val = val_df['answer']
X_test = test_df['prompt']
y_test = test_df['answer']

In [12]:
import pickle
#To save datasets
datasets = {
    'X_train': X_train,
    'y_train': y_train,
    'X_val': X_val,
    'y_val': y_val,
    'X_test': X_test,
    'y_test': y_test,
}

pickle.dump(datasets, open('../../data/dataset-Prompts/datasets.pk', 'wb'))

In [52]:
datasets= pickle.load(open('../../data/dataset-Prompts/datasets.pk', 'rb'))

In [53]:
X_val = datasets['X_val']
y_val = datasets['y_val']

In [54]:
# Device setting
# device = torch.device("mps" if torch.cuda.is_available() else "cpu")
device =  'mps'
# Seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [55]:
# from google.colab import userdata
# from huggingface_hub import login

# token = userdata.get("HF_TOKEN")
# login(token=token)
# !huggingface-cli whoami


# Infererence with Llama model: meta-llama/Llama-3.2-1B

In [71]:
# Upload model
MODEL_ID = "meta-llama/Llama-3.2-1B"
llama_zero = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )
llama_zero.eval()

# Upload tokenizer
tokenizer_llama = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer_llama.pad_token = tokenizer_llama.eos_token

In [72]:
#Model inference LLamma-3.2-1B
def get_outputs(model, prompts, tokenizer, max_new_tokens=2):
    # tokenizer.pad_token = tokenizer.eos_token

    outputs = []

    for i in tqdm(prompts, desc="Generating answers"):
        tokenized_prompt = tokenizer(i, return_tensors='pt').to(device)


        generated_ids = model.generate(
            input_ids=tokenized_prompt['input_ids'],
            attention_mask=tokenized_prompt['attention_mask'],
            max_new_tokens=max_new_tokens,
            do_sample=True,            # enable sampling
            temperature=0.7,           # controls randomness
            top_p=0.9,                 # nucleus sampling
            pad_token_id=tokenizer.eos_token_id
        )

        generated_text = tokenizer.decode(generated_ids[0][tokenized_prompt['input_ids'].shape[1]:], skip_special_tokens=True)
        # print(f'inicio generated_text : {generated_text} fin')

        match = re.search(r'\b([A-D])\b', generated_text.strip().upper())
        outputs.append(match.group(1) if match else "X")

    return outputs

In [73]:
X_val.shape

(83230,)

In [74]:
outputs = get_outputs(llama_zero, X_val, tokenizer_llama)

Generating answers:   4%|██▍                                                       | 3514/83230 [08:44<3:09:46,  7.00it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [75]:
def get_accuracy(outputs, labels):
    matches = sum(p == y for p, y in zip(outputs, labels))
    total = len(labels)
    return matches / total if total > 0 else 0.0

accu = get_accuracy(outputs, y_val)
accu

0.24980175417517722

In [77]:
len(outputs)

83230

# Infererence with Llama model: meta-llama/llama3:8b

In [14]:
# Define model ID
MODEL_ID = "meta-llama/Meta-Llama-3-8B"

# Setup device
device = "mps" if torch.backends.mps.is_available() else "cpu"
dtype = torch.float32  # MPS only supports float32

# Load model
llama_zero = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype,
    device_map={"": device}
)
llama_zero.eval()

# Load tokenizer
tokenizer_llama = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer_llama.pad_token = tokenizer_llama.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [30]:
#Model inference LLamma-3-8B
def get_outputs(model, prompts, tokenizer, max_new_tokens=2):
    torch.manual_seed(42)


    outputs = []

    for i in tqdm(prompts, desc="Generating answers"):
        tokenized_prompt = tokenizer(i, return_tensors='pt').to(device)


        generated_ids = model.generate(
            input_ids=tokenized_prompt['input_ids'],
            attention_mask=tokenized_prompt['attention_mask'],
            max_new_tokens=max_new_tokens,
            do_sample=True,            # enable sampling
            temperature=0.7,           # controls randomness
            top_p=0.9,                 # nucleus sampling
            pad_token_id=tokenizer.eos_token_id
        )

        generated_text = tokenizer.decode(generated_ids[0][tokenized_prompt['input_ids'].shape[1]:], skip_special_tokens=True)
        print(f'inicio generated_text : {generated_text} fin')

        match = re.search(r'\b([A-D])\b', generated_text.strip().upper())
        outputs.append(match.group(1) if match else "X")

    return outputs

In [31]:
outputs = get_outputs(llama_zero, X_val[:20], tokenizer_llama)

Generating answers:   5%|███▎                                                              | 1/20 [00:01<00:33,  1.77s/it]

inicio generated_text :  C
 fin


Generating answers:  10%|██████▌                                                           | 2/20 [00:02<00:22,  1.26s/it]

inicio generated_text :  A

 fin


Generating answers:  15%|█████████▉                                                        | 3/20 [00:03<00:18,  1.10s/it]

...

inicio generated_text :  A
 fin


Generating answers:  95%|█████████████████████████████████████████████████████████████▊   | 19/20 [00:18<00:00,  1.06it/s]

inicio generated_text :  D: fin


Generating answers: 100%|█████████████████████████████████████████████████████████████████| 20/20 [00:18<00:00,  1.06it/s]

inicio generated_text :  A fin


In [10]:
def get_accuracy(outputs, labels):
    matches = sum(p == y for p, y in zip(outputs, labels))
    total = len(labels)
    return matches / total if total > 0 else 0.0

In [33]:
accu = get_accuracy(outputs, y_val[:20])
accu

0.2

In [34]:
for i, (prompt_text, output) in enumerate(zip(X_val[:10], outputs[:10]), 1):
    print(f"--- Prompt {i} ---")
    print(prompt_text)
    print(f"Generated Answer: {output}")
    print("\n")


--- Prompt 1 ---
Instruction: You are a helpful medical assistant. 
Based on the following patient data, answer the multiple-choice question by selecting the correct letter (A, B, C, or D).
Context: This is a cancer patient with the following characteristics:
 * Gender: Female
 * Overal Survival Status: 1:DECEASED
 * Smoking history: Unknown
 * Treatment history: CARBOPLATIN, GEMCITABINE, EXEMESTANE, INVESTIGATIONAL, PACLITAXEL, LAPATINIB
 * Number of tumor diagnoses (ICD-O codes): 1
 * Fraction Genome altered: 0.3578
 * MSI Type: Stable
 * Age of the patient: 34.0
 * Mutations history: CDKN2Ap14ARF, DICER1, JAK1, PAK7, CDKN2A, MLL2, PIK3CA
 * Mutation Count: 7.0
 * Cancer stage: Stage 4
 * History of PDL-1: Not available
 * HER2 status: Yes
Question: "What are the metastatic sites located in the cancer patient's body?"
Options (choose ONLY one option letter): 
 A: Bone, Cns Brain, Intra Abdominal, Liver, Lymph Nodes
 B: Bone, Cns Brain, Liver, Lung, Lymph Nodes
 C: Intra Abdominal, Li

# Infererence with Llama model: meta-llama/Llama-3-8-8B instruct

In [56]:
# # Upload model
# MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
# # Setup device
# device = "mps" if torch.backends.mps.is_available() else "cpu"
# dtype = torch.float32  # MPS only supports float32

# # Load model
# llama_zero = AutoModelForCausalLM.from_pretrained(
#     MODEL_ID,
#     torch_dtype=dtype,
#     device_map={"": device}
# )
# llama_zero.eval()

# # Load tokenizer
# tokenizer_llama = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
# tokenizer_llama.pad_token = tokenizer_llama.eos_token

In [57]:
# print(X_val.iloc[0])

In [58]:
# def get_outputs(model, prompts, tokenizer, max_new_tokens=10):
#     outputs = []

#     for prompt_text in tqdm(prompts, desc="Generating answers"):
#         # Crea el mensaje estilo chat
#         messages = [
#             {"role": "user", "content": prompt_text}
#         ]

#         # Aplica el formato del modelo instruct
#         formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

#         # Tokeniza y pasa a GPU o CPU
#         inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

#         # Generación
#         generated_ids = model.generate(
#             **inputs,
#             max_new_tokens=max_new_tokens,
#             do_sample=True,
#             temperature=0.7,
#             top_p=0.9,
#             pad_token_id=tokenizer.eos_token_id
#         )

#         # Extrae solo la respuesta generada
#         generated_text = tokenizer.decode(
#             generated_ids[0][inputs['input_ids'].shape[1]:],
#             skip_special_tokens=True
#         )
#         print(f'inicio generated_text : {generated_text} fin')


#         # Busca letra A–D como respuesta
#         match = re.search(r'\b([A-D])\b', generated_text.strip().upper())
#         outputs.append(match.group(1) if match else "X")

#     return outputs


In [59]:
# outputs = get_outputs(llama_zero, X_val[:20], tokenizer_llama)

In [60]:
# accu = get_accuracy(outputs, y_val[:20])
# accu

In [61]:
# for i, (prompt_text, output) in enumerate(zip(X_val[:10], outputs[:10]), 1):
#     print(f"--- Prompt {i} ---")
#     print(prompt_text)
#     print(f"Generated Answer: {output}")
#     print("\n")


In [25]:
# #Model inference LLamma-3-8B
# def get_outputs(model, prompts, tokenizer, max_new_tokens=2):

#     outputs = []

#     for i in tqdm(prompts, desc="Generating answers"):
#         tokenized_prompt = tokenizer(i, return_tensors='pt').to(device)


#         generated_ids = model.generate(
#             input_ids=tokenized_prompt['input_ids'],
#             attention_mask=tokenized_prompt['attention_mask'],
#             max_new_tokens=max_new_tokens,
#             do_sample=True,            # enable sampling
#             temperature=0.7,           # controls randomness
#             top_p=0.9,                 # nucleus sampling
#             pad_token_id=tokenizer.eos_token_id
#         )

#         generated_text = tokenizer.decode(generated_ids[0][tokenized_prompt['input_ids'].shape[1]:], skip_special_tokens=True)
#         # print(f'inicio generated_text : {generated_text} fin')

#         match = re.search(r'\b([A-D])\b', generated_text.strip().upper())
#         outputs.append(match.group(1) if match else "X")

#     return outputs

In [87]:
# outputs = get_outputs(llama_zero, X_val[:2000], tokenizer_llama)

Instruction: You are a helpful medical assistant. 
Based on the following patient data, answer the multiple-choice question by selecting the correct letter ([A], [B], [C], or [D]).
Context: This is a cancer patient with the following characteristics:
 * Overal Survival Status: 0:LIVING
 * Cancer stage: Stage 1-3
 * Number of tumor diagnoses (ICD-O codes): 1
 * Fraction Genome altered: 0.1749
 * Gender: Female
 * Mutations history: CBFB, KMT2C, CDH1, ESR1
 * HER2 status: No
 * Mutation Count: 6.0
 * Smoking history: Never
 * History of PDL-1: Not available
 * MSI Type: Stable
 * Age of the patient: 77.0
 * Treatment history: INVESTIGATIONAL, LETROZOLE, INVESTIGATIONAL, PALBOCICLIB, PALBOCICLIB, INVESTIGATIONAL, INVESTIGATIONAL, INVESTIGATIONAL, CAPECITABINE, FULVESTRANT, INVESTIGATIONAL, OLAPARIB, SACITUZUMAB GOVITECAN, ELACESTRANT, DOXORUBICIN LIPOSOMAL, CARBOPLATIN
Question: "What are the metastatic sites located on the patient's body?"
Options (choose ONLY one option letter): 
 [A]: 

# Infererence with Llama model: meta-llama/Llama-3-8-8B instruct - V2

In [8]:
from collections import Counter

In [9]:
datasets= pickle.load(open('../../data/dataset-Prompts/datasets.pk', 'rb'))
X_val = datasets['X_val']
y_val = datasets['y_val']

# Seed for reproducibility
device =  'mps'
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
import transformers
import torch

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
model_id = "meta-llama/Llama-3.2-1B"
# model_id = "meta-llama/Llama-4-Scout-17B-16E-Instruct"
# model_id = "meta-llama/Llama-4-Maverick-17B-128E-Instruct"
# model_id = "google/medgemma-4b-it"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.

KeyboardInterrupt



KeyboardInterrupt: 

In [12]:
system = "You are a helpful medical assistant. Based on the following patient data, answer the multiple-choice question by selecting onlh and only the correct letter ([A], [B], [C], or [D])."
out = []
for prompt_text in tqdm(X_val[:100]): 
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt_text},
    ]
    
    outputs = pipeline(
        messages,
        max_new_tokens=20,
        pad_token_id=128001
    )
    opt = '[X]'
    for option in ['[A]', '[B]', '[C]', '[D]']:
        if option in outputs[0]["generated_text"][-1]['content']:
            opt = option
    
    out.append(opt)

100%|███████████████████████████████████████████████████████████████████████████████████| 100/100 [01:57<00:00,  1.18s/it]


In [13]:
get_accuracy(out, y_val[:100])

0.43

In [14]:
Counter(out)

Counter({'[C]': 32, '[D]': 27, '[X]': 18, '[B]': 16, '[A]': 7})

# MedGEmma Instruction

In [4]:
from collections import Counter

In [5]:
datasets= pickle.load(open('../../data/dataset-Prompts/datasets.pk', 'rb'))
X_val = datasets['X_val']
y_val = datasets['y_val']

# Seed for reproducibility
device =  'mps'
torch.manual_seed(42)
np.random.seed(42)

In [10]:
import transformers
import torch

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
# model_id = "meta-llama/Llama-3.2-1B"
# model_id = "meta-llama/Llama-4-Scout-17B-16E-Instruct"
# model_id = "meta-llama/Llama-4-Maverick-17B-128E-Instruct"
# model_id = "google/medgemma-4b-it"
model_id = "google/medgemma-27b-text-it"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

model-00008-of-00011.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00011.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

...

{"timestamp":"2025-07-05T01:42:43.540068Z","level":"WARN","fields":{"message":"Reqwest(reqwest::Error { kind: Request, url: \"https://transfer.xethub.hf.co/xorbs/default/429826e75a6855c627678a8cbc48f88419fb16a9b174f2e5db8f539adb11bfc8?X-Xet-Signed-Range=bytes%3D0-66998988&Expires=1751683332&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly90cmFuc2Zlci54ZXRodWIuaGYuY28veG9yYnMvZGVmYXVsdC80Mjk4MjZlNzVhNjg1NWM2Mjc2NzhhOGNiYzQ4Zjg4NDE5ZmIxNmE5YjE3NGYyZTVkYjhmNTM5YWRiMTFiZmM4P1gtWGV0LVNpZ25lZC1SYW5nZT1ieXRlcyUzRDAtNjY5OTg5ODgiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NTE2ODMzMzJ9fX1dfQ__&Signature=kGWJdjeZQDV-cIHdrhIAoXAj1EEWgMDwB1w3NllwX6Lm9Nf6PDvthxOISiFyLw~0rXft9dZtKAf4nD7TD9H4dWtc73Ch~3MqLpVeeMEdas1ErSnrrUT0b9LPkyKNwtXVxw24e2242UCgBbBd~WtZNDtWx93WXQo-76Ugkc2DLSts7BP6xmNOnmzOZcdcjePGJ-B1WOK7gfzt4KydjfSgjDhwppgGiAWZutty1dqaFrifnqsGBXzpYQyib4LJhwI~nvgVDnoy8CruNNGQFKt1j1niNWReqRAuz0FkX61xVsTFunCjK4CrNBbpd-JCVDEUXwADudT2W-wyVMh2~GL5EQ__&Key-Pair-Id=K2L8F4GPSG1IFC\",

model-00009-of-00011.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

{"timestamp":"2025-07-05T01:42:55.146614Z","level":"WARN","fields":{"message":"Reqwest(reqwest::Error { kind: Request, url: \"https://transfer.xethub.hf.co/xorbs/default/795f14a7f21f0d7b7894434d1004161654f84205d2275f183c09c9955a0fb6d6?X-Xet-Signed-Range=bytes%3D0-58263798&Expires=1751683334&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly90cmFuc2Zlci54ZXRodWIuaGYuY28veG9yYnMvZGVmYXVsdC83OTVmMTRhN2YyMWYwZDdiNzg5NDQzNGQxMDA0MTYxNjU0Zjg0MjA1ZDIyNzVmMTgzYzA5Yzk5NTVhMGZiNmQ2P1gtWGV0LVNpZ25lZC1SYW5nZT1ieXRlcyUzRDAtNTgyNjM3OTgiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NTE2ODMzMzR9fX1dfQ__&Signature=dDbsiLVWkOK4upiC~Yyd16fEMu5FJMWtSTesMNbXHxYiUPKBYl0axinn1ttZy-tJun9vQ7G2l66q-WOYr8Yr4VeJB9dq18s5ZL0Iz0OVBpFjxJCj5YCMfo5yszD3m-Bm20BdG1EKcYBSqRBQbfKZPGPL7a1VvfpDnsX1jpYJD5EVMnyS9L1yhcyF08L2Mx6AhuUXYSf~9k-8SwTyqXJ~NBT0mjfzRAb55Nl9-rvHjjiZxo7NKrzri-gQ5HhoGNqJM0BDSiTGPmb6ZJ~QVtvpALEax6SyScZaZ53uV3NdMhWcDOjC2hz-6148hk2yop~90GRPCfhvVfkHmsSciPv9aA__&Key-Pair-Id=K2L8F4GPSG1IFC\",


KeyboardInterrupt



KeyboardInterrupt: 

In [7]:
system = "You are a helpful medical assistant. Based on the following patient data, answer the multiple-choice question by selecting onlh and only the correct letter ([A], [B], [C], or [D])."
out = []
for prompt_text in tqdm(X_val[:100]): 
    messages = [
        {"role": "system", "content": [{"type": "text", "text": system}]},
        {"role": "user", "content": [{"type": "text", "text": prompt_text}]},
    ]
    
    outputs = pipeline(
        messages,
        max_new_tokens=20,
        # pad_token_id=128001
    )
    opt = '[X]'
    for option in ['[A]', '[B]', '[C]', '[D]']:
        if option in outputs[0]["generated_text"][-1]['content']:
            opt = option
    
    out.append(opt)

100%|███████████████████████████████████████████████████████████████████████████████████| 100/100 [01:30<00:00,  1.11it/s]


In [8]:
get_accuracy(out, y_val[:100])

0.51

In [9]:
Counter(out)

Counter({'[A]': 47, '[B]': 19, '[C]': 18, '[D]': 16})

# OpenAI 

In [43]:
from dataclasses import dataclass
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext
from typing import List, Dict
import yaml
import os
import pandas as pd
import time 

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = '' 
os.environ['GEMINI_API_KEY'] = ''

In [22]:
system = "You are a helpful medical assistant. Based on the following patient data, answer the multiple-choice question by selecting onlh and only the correct letter ([A], [B], [C], or [D])."
class Output(BaseModel):
    answer: str = Field(description="returns the answer to the questions as one of: [A], [B], [C] or [D]")
    

In [92]:
agent = Agent(
    'gemini-2.5-flash',
    # 'openai:gpt-4.1-nano-2025-04-14',
    # 'openai:gpt-4.1-mini-2025-04-14',
    # 'openai:gpt-4.1',
    system_prompt=system,
    output_type=Output,
    Instrument=True
)

In [93]:
out = []
for prompt_text in tqdm(X_val[:100]): 
    result = await agent.run(prompt_text)
    out.append(result.output.answer)
    time.sleep(0.5)

100%|███████████████████████████████████████████████████████████████████████████████████| 100/100 [21:14<00:00, 12.74s/it]


In [94]:
out = [f'[{i}]' if not '[' in i else i for i in out ]

In [96]:
get_accuracy(out, y_val[:100])

0.64

# Results

Performance on the 100 first questions: 

* GPT4.1-nano: 0.5
* GPT4.1-mini: 0.56
* GPT4.1: 0.64
* Llama-3-8B-Instruct: 0.35
* Llama-3.2-1B-Instruct: 0.43
* Llama-3.2-3B-Instruct: 0.47
* Gemini-2.5-flash: 0.64
* medgemma-4b-it: 0.51

In [2]:
import transformers
transformers.__version__

'4.52.4'

In [3]:
import transformers
transformers.__version__

'4.52.4'